In [11]:
import os
import random
import shutil
from osgeo import gdal, ogr
import torch
import geopandas as gpd
from torchvision import transforms
from torch.utils.data import DataLoader
import segmentation_models_pytorch as smp
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2
from geonet import tiler, mask, raster, dataset
from geonet.visualizations import plotImagePair
from PIL import Image

In [6]:
src_image = "../../data/_mvp/liski_jj_20-3857.tif"
src_vector = "../../data/_mvp/belgorod_train.shp"
tiles_dir = "../../data/_mvp/agro20jj_train_tiles"
labels_dir = "../../data/_mvp/agro_train_labels"

In [7]:
raster_tiler = tiler.RasterTiler(dest_dir=tiles_dir, src_tile_size=(512, 512), tile_overlay=32, verbose=True)

Initializing Tiler...
Tiler initialized.
dest_dir: ../../data/_mvp/agro20jj_train_tiles
dest_crs will be inferred from source data.
src_tile_size: (512, 512)
tile size units metric: False
Resampling is set to None


In [8]:
raster_bounds_crs = raster_tiler.tile(src_image)

Beginning tiling...


Checking input data...
[1, 2, 3, 4]
Source CRS: EPSG:3857
Destination CRS: EPSG:3857
Inputs OK.
count of tile bounds 576

Tiling complete. Cleaning up...
Done. CRS returned for vector tiling.


In [70]:
vector_tiler = tiler.VectorTiler(dest_dir=labels_dir, verbose=True)
vector_tiler.tile(src_vector,
                  tile_bounds=raster_tiler.tile_bounds,
                  tile_bounds_crs=raster_bounds_crs)

Preparing the tiler...
Initialization done.


Num tiles: 304



In [6]:
def add_prefix(path, prefix):
    files = os.listdir(path)
    for index, file in enumerate(files):
        os.rename(os.path.join(path, file), os.path.join(path, prefix+file))

In [71]:
add_prefix("../../data/_mvp/agro_train_labels/bel_v/", "bel_v")

In [34]:
images = os.listdir("../../data/_mvp/agro20jj_train_tiles/")
#labels = os.listdir("../../data/_mvp/agro19_train_labels/")
masks = os.listdir("../../data/_mvp/agro20jj_train_masks/")
images.sort()
#labels.sort()
masks.sort()
print(len(images))
#print(len(labels))
print(len(masks))

1826
1826


In [31]:
images[0:5]

['borisoglebsk_jj_20-3857_4660971_6679049.tif',
 'borisoglebsk_jj_20-3857_4660971_6686803.tif',
 'borisoglebsk_jj_20-3857_4660971_6694556.tif',
 'borisoglebsk_jj_20-3857_4660971_6702310.tif',
 'borisoglebsk_jj_20-3857_4660971_6710064.tif']

In [52]:
labels[0:5]

['bel_trgeoms_4020878_6524396.geojson',
 'bel_trgeoms_4020878_6531968.geojson',
 'bel_trgeoms_4020878_6539541.geojson',
 'bel_trgeoms_4020878_6547113.geojson',
 'bel_trgeoms_4020878_6554686.geojson']

In [32]:
masks[0:5]

['.ipynb_checkpoints',
 'borisoglebsk_jj_20-3857_4660971_6679049.tif',
 'borisoglebsk_jj_20-3857_4660971_6686803.tif',
 'borisoglebsk_jj_20-3857_4660971_6694556.tif',
 'borisoglebsk_jj_20-3857_4660971_6702310.tif']

In [37]:
import shutil
shutil.rmtree("../../data/_mvp/agro20jj_test_tiles/.ipynb_checkpoints/")

In [104]:
for i in range(0, len(images)):
    gdf = gpd.read_file(labels_dir + '/' + labels[i])
    ref_im = tiles_dir + '/' + images[i]
    fp_mask = mask.footprint_mask(gdf, reference_im=ref_im, do_transform="reference_im")
    #b_mask = mask.boundary_mask(fp_mask, boundary_width=5)
    try:
        #c_mask = mask.contact_mask(gdf, reference_im=ref_im, contact_spacing=5)
        #mask_full = np.dstack((fp_mask, b_mask, c_mask))
        mask_im = "../../data/_mvp/agro_train_masks/" + images[i]
        Image.fromarray(fp_mask).save(mask_im)
    except:
        os.remove("../../data/_mvp/agro_train_labels/" + labels[i])
        os.remove("../../data/_mvp/agro_train_tiles/" + images[i])
        print(labels[i] + " processing failed. Files removed")

In [18]:
# split files on train and test
def test_split(train_dir, train_masks_dir, test_dir, test_masks_dir, shape, per=0.3):
    k = np.int(np.round(shape * per))
    test_idx = random.sample(range(0, shape), k)
    file_names = os.listdir(train_dir)
    i = 0
    for file_name in file_names:
        if i in test_idx:
            shutil.move(os.path.join(train_dir, file_name), test_dir)
            shutil.move(os.path.join(train_masks_dir, file_name), test_masks_dir)
        i+=1

In [29]:
for count, filename in enumerate(os.listdir("../../data/_mvp/agro20jj_train_masks/v/")): 
    dst = filename.split("_")
    del dst[0:2]
    dst = '_'.join(dst)
    dst = 'vrn_20-07_'+dst
    src ="../../data/_mvp/agro20jj_train_masks/v/" + filename 
    dst ="../../data/_mvp/agro20jj_train_masks/v/" + dst 
          
    # rename() function will 
    # rename all the files 
    os.rename(src, dst)

In [35]:
test_split("../../data/_mvp/agro20jj_train_tiles/", "../../data/_mvp/agro20jj_train_masks/",
          "../../data/_mvp/agro20jj_test_tiles/", "../../data/_mvp/agro20jj_test_masks/", shape=1826, per=0.3)

# Create Dataset

In [57]:
# add augmentations
aug = A.Compose([
    #A.Normalize(mean=(0.0095, 0.0087, 0.0078), std=(0.0075, 0.0070, 0.0060)),
    A.RandomRotate90(p=0.4),
    A.Flip(p=0.4),
    A.GridDistortion(p=0.4),
    #A.Downscale(scale_min=0.2, scale_max=0.3, p=0.3),
    A.Transpose(p=0.4),
    #A.Lambda(image=preprocess_input),
    A.Lambda(image=to_tensor),
])

In [38]:
def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1).astype('float32')

def preprocess_input(x, mean=[0.485, 0.456, 0.406, 0.405], std=[0.229, 0.224, 0.225, 0.225], input_space="RGB", input_range=[0,1], **kwargs):

    if input_space == "BGR":
        x = x[..., ::-1].copy()

    if input_range is not None:
        if x.max() > 1 and input_range[1] == 1:
            x = x / 255.0

    if mean is not None:
        mean = np.array(mean)
        x = x - mean

    if std is not None:
        std = np.array(std)
        x = x / std

    return x


def get_preprocessing(preprocessing_fn):
    """Construct preprocessing transform
    
    Args:
        preprocessing_fn (callbale): data normalization function 
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose
    
    """
    
    _transform = [
        A.RandomRotate90(p=0.4),
        A.Flip(p=0.4),
        A.GridDistortion(p=0.4),
        #A.Downscale(scale_min=0.2, scale_max=0.3, p=0.3),
        A.Transpose(p=0.4),
        A.Lambda(image=preprocess_input),
        A.Lambda(image=to_tensor),
    ]
    return A.Compose(_transform)

In [39]:
def get_inference_preprocessing(preprocessing_fn):
    """Construct preprocessing transform
    
    Args:
        preprocessing_fn (callbale): data normalization function 
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose
    
    """
    
    _transform = [
        A.Lambda(image=preprocess_input),
        A.Lambda(image=to_tensor)
    ]
    return A.Compose(_transform)

In [40]:
# initialize model
ENCODER = 'se_resnext50_32x4d'
ENCODER_WEIGHTS = 'imagenet'
CLASSES = ['agro']
ACTIVATION = 'sigmoid' # could be None for logits or 'softmax2d' for multicalss segmentation
DEVICE = 'cuda'

# create segmentation model with pretrained encoder
model = smp.FPN(
    encoder_name=ENCODER, 
    classes=len(CLASSES), 
    activation=ACTIVATION,
    in_channels=4,
)

preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)

In [41]:
trainData = dataset.TiledRasterDataset("../../data/_mvp/agro20jj_train_tiles/", "../../data/_mvp/agro20jj_train_masks/", classes=["agro"], preprocessing=get_preprocessing(preprocessing_fn))
validData = dataset.TiledRasterDataset("../../data/_mvp/agro20jj_test_tiles/", "../../data/_mvp/agro20jj_test_masks/", classes=["agro"], preprocessing=get_preprocessing(preprocessing_fn))

/home/DVL/glotovaa/projects/data_science/lib/python3.6/site-packages/albumentations/augmentations/transforms.py:2908: UserWarning: Using lambda is incompatible with multiprocessing. Consider using regular functions or partial().
  "Using lambda is incompatible with multiprocessing. "


In [42]:
'''x_train_file = raster.get_array_from_tiff("../../data/_mvp/liski_3857.tif")
x_train_file = np.dstack(x_train_file)
y_train_file = raster.get_array_from_tiff("../../data/_mvp/liski_agro_mask.tif")[0]
#x_valid_file = raster.get_array_from_tiff("../../data/_mvp/vrn_train_3857.tif")
#x_valid_file = np.dstack(x_valid_file)
#y_valid_file = raster.get_array_from_tiff("../../data/_mvp/vrn_3857_mask.tif")[0]
trainData2 = dataset.RasterDataset(x_train_file.astype(float), y_train_file.astype(float), classes=["agro"], preprocessing=get_preprocessing(preprocessing_fn))
validData2 = dataset.RasterDataset(x_train_file.astype(float), y_train_file.astype(float), classes=["agro"], preprocessing=get_preprocessing(preprocessing_fn))
'''

'x_train_file = raster.get_array_from_tiff("../../data/_mvp/liski_3857.tif")\nx_train_file = np.dstack(x_train_file)\ny_train_file = raster.get_array_from_tiff("../../data/_mvp/liski_agro_mask.tif")[0]\n#x_valid_file = raster.get_array_from_tiff("../../data/_mvp/vrn_train_3857.tif")\n#x_valid_file = np.dstack(x_valid_file)\n#y_valid_file = raster.get_array_from_tiff("../../data/_mvp/vrn_3857_mask.tif")[0]\ntrainData2 = dataset.RasterDataset(x_train_file.astype(float), y_train_file.astype(float), classes=["agro"], preprocessing=get_preprocessing(preprocessing_fn))\nvalidData2 = dataset.RasterDataset(x_train_file.astype(float), y_train_file.astype(float), classes=["agro"], preprocessing=get_preprocessing(preprocessing_fn))\n'

In [43]:
#trainData[0]

In [44]:
# add data loaders
train_loader = DataLoader(trainData, batch_size=1, shuffle=True, num_workers=3)
valid_loader = DataLoader(validData, batch_size=1, shuffle=True, num_workers=2)

In [45]:
loss = smp.utils.losses.DiceLoss()
metrics = [
    smp.utils.metrics.IoU(threshold=0.5),
]

optimizer = torch.optim.AdamW([ 
    dict(params=model.parameters(), lr=0.00015),
])

In [46]:
train_epoch = smp.utils.train.TrainEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    optimizer=optimizer,
    device = DEVICE,
)

valid_epoch = smp.utils.train.ValidEpoch(
    model, 
    loss=loss, 
    metrics=metrics,
    device = DEVICE
)

In [47]:
model.cuda()

FPN(
  (encoder): SENetEncoder(
    (layer0): Sequential(
      (conv1): Conv2d(4, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace=True)
      (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    )
    (layer1): Sequential(
      (0): SEResNeXtBottleneck(
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu)

In [48]:
# train model
max_score = 0.78
'''checkpoint = torch.load("./weights/best/agronet_chekpoint_v1.3 - 0.7770444019770664.pth")
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']'''

for i in range(0, 80):
    
    print('\nEpoch: {}'.format(i))
    train_logs = train_epoch.run(train_loader)
    valid_logs = valid_epoch.run(valid_loader)
    
    # do something (save model, change lr, etc.)
    if max_score < valid_logs['iou_score']:
        max_score = valid_logs['iou_score']
        torch.save(model, './weights/agronet-20_07_v1.3 - {}.pth'.format(max_score))
        torch.save({
            'epoch': i,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, './weights/agronet-20_07_v1.3-chekpoint - {}.pth'.format(max_score))
        print('Model saved!')
        
    if i == 15:
        optimizer.param_groups[0]['lr'] = 7e-5
        print('Decrease decoder learning rate to 7e-5!')


Epoch: 0
valid: 100%|██████████| 548/548 [00:13<00:00, 39.75it/s, dice_loss - 0.1785, iou_score - 0.7456]

Epoch: 1
valid: 100%|██████████| 548/548 [00:13<00:00, 39.28it/s, dice_loss - 0.2093, iou_score - 0.7667]

Epoch: 2
valid: 100%|██████████| 548/548 [00:13<00:00, 39.25it/s, dice_loss - 0.2375, iou_score - 0.7452]

Epoch: 3
valid: 100%|██████████| 548/548 [00:13<00:00, 39.29it/s, dice_loss - 0.1611, iou_score - 0.7769]

Epoch: 4
valid: 100%|██████████| 548/548 [00:13<00:00, 39.28it/s, dice_loss - 0.1575, iou_score - 0.7786]

Epoch: 5
valid: 100%|██████████| 548/548 [00:13<00:00, 39.22it/s, dice_loss - 0.6832, iou_score - 0.247] 

Epoch: 6
valid: 100%|██████████| 548/548 [00:13<00:00, 39.23it/s, dice_loss - 0.1657, iou_score - 0.7641]

Epoch: 7
valid: 100%|██████████| 548/548 [00:13<00:00, 39.19it/s, dice_loss - 0.2365, iou_score - 0.706] 

Epoch: 8
valid: 100%|██████████| 548/548 [00:13<00:00, 39.18it/s, dice_loss - 0.1655, iou_score - 0.7784]

Epoch: 9
valid: 100%|██████████| 548

In [49]:
# model's inference
    best_model = torch.load('../geonet/weights/agronet-20_07_v1.3 - 0.8182464988102082.pth')

# Inference

In [88]:
del(img, mas, test_loader)

In [50]:
img = raster.get_array_from_tiff("../../data/_mvp/vrn_19_3857.tif")
img = np.dstack(img)

In [51]:
mas = raster.get_array_from_tiff("../../data/_mvp/vrn_3857_mask.tif")[0]

In [52]:
testData = dataset.RasterDataset(img, mas, classes=['agro'], step=384, preprocessing=get_inference_preprocessing(preprocessing_fn))

/home/DVL/glotovaa/projects/data_science/lib/python3.6/site-packages/albumentations/augmentations/transforms.py:2908: UserWarning: Using lambda is incompatible with multiprocessing. Consider using regular functions or partial().
  "Using lambda is incompatible with multiprocessing. "


In [54]:
test_loader = DataLoader(testData, batch_size=1, shuffle=False, num_workers=1)

In [55]:
ext_x = np.zeros(shape=(img.shape[0], img.shape[1]), dtype=np.float32)
step = 384
tile_size = 512
xc = round(img.shape[0] / step) + 1
yc = round(img.shape[1] / step) + 1

i = 0
for batch in test_loader:
    m = i % xc
    j = i // xc
    #x_tensor = torch.from_numpy(batch[0]).unsqueeze(0)
    pr_mask = best_model.predict(batch[0].cuda())
    pr_mask = (pr_mask.cpu().numpy().round(3))
    '''if i < 5:
        plotImagePair(
        image=batch[0][0].reshape(512, 512, 4), 
        #ground_truth_mask=gt_mask, 
        predicted_mask=pr_mask
    )'''
            
    if (step*m+tile_size) > img.shape[0]:
        if (step*j+tile_size) > img.shape[1]:
            ext_x[(img.shape[0]-tile_size):img.shape[0], (img.shape[1]-tile_size):img.shape[1]] = np.maximum(ext_x[(img.shape[0]-tile_size):img.shape[0], (img.shape[1]-tile_size):img.shape[1]], pr_mask)
        else:
            ext_x[(img.shape[0]-tile_size):img.shape[0], step*j:(step*j+tile_size)] = np.maximum(ext_x[(img.shape[0]-tile_size):img.shape[0], step*j:(step*j+tile_size)], pr_mask)
    elif (step*j+tile_size) > img.shape[1]:
        ext_x[step*m:(step*m+tile_size), (img.shape[1]-tile_size):img.shape[1]] = np.maximum(ext_x[step*m:(step*m+tile_size), (img.shape[1]-tile_size):img.shape[1]], pr_mask)
    else:
        ext_x[step*m:(step*m+tile_size), step*j:(step*j+tile_size)] = np.maximum(ext_x[step*m:(step*m+tile_size), step*j:(step*j+tile_size)], pr_mask)
    
    i += 1

raster.get_raster_from_array(ext_x, "../../data_science/mvp/vrn_20-07_agro_predicted_v.1.3.tif", "../../data/_mvp/vrn_train_3857.tif")

# Vectorization of results

In [ ]:
sourceRaster = gdal.Open("../agronet/data/kursk_predicted.tif")
band = sourceRaster.GetRasterBand(1)
driver = ogr.GetDriverByName("ESRI Shapefile")
outDatasource = driver.CreateDataSource("../agronet/data/kursk_vector.shp")
outLayer = outDatasource.CreateLayer("polygonized", srs=None, OGRwkbGeometryType)
newField = ogr.FieldDefn('MYFLD', ogr.OFTInteger)
outLayer.CreateField(newField)
gdal.Polygonize(band, None, outLayer, 1, [], callback=None )

# Vector post-processing